In [ ]:
!pip install ipython-sql
!pip install sqlite3

In [3]:
import fetch_data
df = fetch_data.fromCSV()
df.head(3)

INFO:fetch_data:Fetching file by url on 2025-02-15 Time 21:07:24+00:00 UTC


,kjønn,utdanning,erfaring,arbeidssted,arbeidssituasjon,fag,lønn,bonus?
0,mann,4,7,Agder,"in-house, privat sektor",AI / maskinlæring,865000,Nei
1,mann,5,2,Nordland,"in-house, offentlig/kommunal sektor",AI / maskinlæring,756000,Nei
2,mann,4,30,Oslo,konsulent,AI / maskinlæring,1500000,Nei


In [4]:
#переименовываем колонки, т.к. имена колонок на норвежском
df = df.rename(columns = {'kjønn':'sex',
                          'utdanning':'education',
                          'erfaring':'experience',
                          'arbeidssted':'workplace',
                          'arbeidssituasjon':'work_situation',
                          'fag':'subject',
                          'lønn':'salary'
                         })
df.head(3)

,sex,education,experience,workplace,work_situation,subject,salary,bonus?
0,mann,4,7,Agder,"in-house, privat sektor",AI / maskinlæring,865000,Nei
1,mann,5,2,Nordland,"in-house, offentlig/kommunal sektor",AI / maskinlæring,756000,Nei
2,mann,4,30,Oslo,konsulent,AI / maskinlæring,1500000,Nei


In [5]:
import sqlite3
data_base_name = 'hw_03_data_base.db'
con = sqlite3.connect(data_base_name)

In [6]:
table_name = 'data'
df.to_sql(table_name, con=con, if_exists='replace')

2682

In [8]:
cur = con.cursor()
print("LIMIT by SQL")
for row in cur.execute(f"SELECT * FROM {table_name} LIMIT 3"):
    print(row)


print("===============")
print("LIMIT by Python - bad")
i=0
print_rows_count = 3
for row in cur.execute(f"SELECT * FROM {table_name}"):
    print(row)
    i+=1
    if i==print_rows_count or i>print_rows_count :
        break;

LIMIT by SQL
(0, 'mann', 4, 7, 'Agder', 'in-house, privat sektor', 'AI / maskinlæring', 865000, 'Nei')
(1, 'mann', 5, 2, 'Nordland', 'in-house, offentlig/kommunal sektor', 'AI / maskinlæring', 756000, 'Nei')
(2, 'mann', 4, 30, 'Oslo', 'konsulent', 'AI / maskinlæring', 1500000, 'Nei')
LIMIT by Python - bad
(0, 'mann', 4, 7, 'Agder', 'in-house, privat sektor', 'AI / maskinlæring', 865000, 'Nei')
(1, 'mann', 5, 2, 'Nordland', 'in-house, offentlig/kommunal sektor', 'AI / maskinlæring', 756000, 'Nei')
(2, 'mann', 4, 30, 'Oslo', 'konsulent', 'AI / maskinlæring', 1500000, 'Nei')


In [9]:
#посчитаем количество людей в БД с разбивкой по полу и месту работы 
res = cur.execute(f"""
            SELECT sex,
            workplace,
            COUNT(sex)
            FROM {table_name}
            GROUP BY sex, workplace
        """)
res.fetchall()

[('annet / ønsker ikke oppgi', 'Innlandet', 2),
 ('annet / ønsker ikke oppgi', 'Nordland', 1),
 ('annet / ønsker ikke oppgi', 'Oslo', 12),
 ('annet / ønsker ikke oppgi', 'Rogaland', 1),
 ('annet / ønsker ikke oppgi', 'Vestland', 2),
 ('annet / ønsker ikke oppgi', 'Viken', 1),
 ('kvinne', 'Agder', 6),
 ('kvinne', 'Innlandet', 6),
 ('kvinne', 'Møre og Romsdal', 4),
 ('kvinne', 'Nordland', 4),
 ('kvinne', 'Oslo', 250),
 ('kvinne', 'Rogaland', 17),
 ('kvinne', 'Troms og Finnmark', 4),
 ('kvinne', 'Trøndelag', 21),
 ('kvinne', 'Vestfold og Telemark', 11),
 ('kvinne', 'Vestland', 25),
 ('kvinne', 'Viken', 17),
 ('kvinne', 'utlandet', 3),
 ('mann', 'Agder', 88),
 ('mann', 'Innlandet', 58),
 ('mann', 'Møre og Romsdal', 43),
 ('mann', 'Nordland', 40),
 ('mann', 'Oslo', 1215),
 ('mann', 'Rogaland', 144),
 ('mann', 'Troms og Finnmark', 32),
 ('mann', 'Trøndelag', 175),
 ('mann', 'Vestfold og Telemark', 100),
 ('mann', 'Vestland', 231),
 ('mann', 'Viken', 144),
 ('mann', 'utlandet', 25)]

In [10]:
#загружаем в DataFrame результат выполнения запроса к БД. Колонке в результате запроса будут с другими именами
import pandas as pd
df = pd.read_sql_query(f"""
            SELECT sex as 'Sex',
            workplace as 'Place',
            COUNT(sex) as 'Count'
            FROM {table_name} 
            GROUP BY sex, workplace
        """, con)
df.head(5)

,Sex,Place,Count
0,annet / ønsker ikke oppgi,Innlandet,2
1,annet / ønsker ikke oppgi,Nordland,1
2,annet / ønsker ikke oppgi,Oslo,12
3,annet / ønsker ikke oppgi,Rogaland,1
4,annet / ønsker ikke oppgi,Vestland,2


In [11]:
#в БД меняем значения в колонке sex с норвежского на английский
def update_text_in_column(cur:sqlite3.Cursor,
                          table_name: str,
                          column_name: str,
                          current_value: str,
                          new_value: str):
    res = cur.execute(f"""
            UPDATE {table_name}
            SET {column_name} = '{new_value}'
            WHERE {column_name} = '{current_value}'
        """)
    res.fetchall()
    None

update_text_in_column(cur, table_name, 'sex', 'mann', 'male' )
update_text_in_column(cur, table_name, 'sex', 'kvinne', 'female' )
update_text_in_column(cur, table_name, 'sex', 'annet / ønsker ikke oppgi', 'other / do not wish to state' )

In [12]:
#посчитаем количество людей в БД с разбивкой по полу и месту работы 
res = cur.execute(f"""
            SELECT sex,
            workplace,
            COUNT(sex)
            FROM {table_name}
            GROUP BY sex, workplace
        """)
res.fetchall()

[('female', 'Agder', 6),
 ('female', 'Innlandet', 6),
 ('female', 'Møre og Romsdal', 4),
 ('female', 'Nordland', 4),
 ('female', 'Oslo', 250),
 ('female', 'Rogaland', 17),
 ('female', 'Troms og Finnmark', 4),
 ('female', 'Trøndelag', 21),
 ('female', 'Vestfold og Telemark', 11),
 ('female', 'Vestland', 25),
 ('female', 'Viken', 17),
 ('female', 'utlandet', 3),
 ('male', 'Agder', 88),
 ('male', 'Innlandet', 58),
 ('male', 'Møre og Romsdal', 43),
 ('male', 'Nordland', 40),
 ('male', 'Oslo', 1215),
 ('male', 'Rogaland', 144),
 ('male', 'Troms og Finnmark', 32),
 ('male', 'Trøndelag', 175),
 ('male', 'Vestfold og Telemark', 100),
 ('male', 'Vestland', 231),
 ('male', 'Viken', 144),
 ('male', 'utlandet', 25),
 ('other / do not wish to state', 'Innlandet', 2),
 ('other / do not wish to state', 'Nordland', 1),
 ('other / do not wish to state', 'Oslo', 12),
 ('other / do not wish to state', 'Rogaland', 1),
 ('other / do not wish to state', 'Vestland', 2),
 ('other / do not wish to state', 'Vik

In [17]:
con.close()